In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

import itertools

from cpd_utils import *

import time
import bisect

import pandas as pd

In [103]:
class dp_cv_random_covariance(dp_cv_random):
    def __init__(self, grid_n, repeat, lam_list, gamma_list, smooth = 100, buffer = 200):
        self.grid_n = grid_n
        self.repeat = repeat
        self.lam_list = lam_list
        self.gamma_list = gamma_list
        self.smooth = smooth
        self.buffer = buffer
    
    def estimate_func(self, Y, lam):
        return Y.T @ Y / Y.shape[0]

    def loss_func(self, Y, sig):
        '''
        currently the penalty is not included in the value of loss
        '''
        if np.linalg.det(sig) < 1e-7:
            sig_reg = sig + 0.1 * np.eye(Y.shape[1])
        else:
            sig_reg = sig.copy()
        omega = np.linalg.inv(sig_reg)
        return np.trace(omega @ (Y.T @ Y)) + Y.shape[0] * np.log(np.linalg.det(sig_reg))

class dcdp_cv_random_covariance(dcdp_cv_random):
    def __init__(self, grid_n, repeat, lam_list, gamma_list, smooth = 100, 
                 buffer = 200, step_refine = 1, buffer_refine = 30, lam_refine = 0.1):
        self.grid_n = grid_n
        self.repeat = repeat
        self.lam_list = lam_list
        self.gamma_list = gamma_list
        
        self.smooth = smooth
        self.buffer = buffer
        
        self.step_refine = step_refine
        self.buffer_refine = buffer_refine
        self.lam_refine = lam_refine

    def joint_estimate_func(self, Y1, Y2, lam):
        """
        may also try group lasso
        """
        beta_hat1 = self.estimate_func(Y1, lam)
        
        beta_hat2 = self.estimate_func(Y2, lam)
        return beta_hat1, beta_hat2
        
    def estimate_func(self, Y, lam):
        return Y.T @ Y / Y.shape[0]

    def loss_func(self, Y, sig):
        '''
        currently the penalty is not included in the value of loss
        '''
        if np.linalg.det(sig) < 1e-7:
            sig_reg = sig + 0.1 * np.eye(Y.shape[1])
        else:
            sig_reg = sig.copy()
        omega = np.linalg.inv(sig_reg)
        return np.trace(omega @ (Y.T @ Y)) + Y.shape[0] * np.log(np.linalg.det(sig_reg))
    
    def fit_with_cp(self, Y, cp, lam = 0.1):
        if len(Y.shape) == 1:
            Y = Y.reshape((-1, 1))
        K = len(cp) - 1
        p = Y.shape[1]
        beta_path = np.zeros((K, p, p))

        for i in range(K):
            Y_cur = Y[cp[i]:cp[i + 1]]
            beta_cur = self.estimate_func(Y_cur, lam)
            beta_path[i] = beta_cur.copy()    

        return beta_path

In [126]:
def generate_data_covariance(n, T, theta):
    p = theta.shape[1]
    y_train_joint = np.concatenate([
        np.random.multivariate_normal(mean = np.zeros(p), cov = theta[i], size = n[i]) 
        for i in range(T)], axis = 0)
    nt = len(y_train_joint)
    
    return nt, y_train_joint

# DCDP

Signal too weak

In [118]:
T = 4
Delta = 500
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 5
theta = np.zeros((T, p, p))
theta[0] = np.eye(p)

rho = 0.1
theta[1] = (1 - rho) * np.eye(p) + rho * np.ones((p, p))

rho = -0.1
theta[2] = (1 - rho) * np.eye(p) + rho * np.ones((p, p))

theta[3] = np.eye(p)

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5

nt, Y_train = generate_data_covariance(n, T, theta)
nt, Y_test = generate_data_covariance(n, T, theta)

In [119]:
theta.shape
Y_train.shape

(2000, 5)

In [124]:
grid_n = 100
repeat = 1
gamma_list = [5, 20, 50]
lam_list = [0]

B = 1
run_time_dc = np.zeros(B)
loc_error_dc = np.zeros(B)

print('---------- divide and conquer -----------')
for b in range(B):
    start_time = time.time()
    dcdp = dcdp_cv_random_covariance(grid_n, repeat, lam_list, gamma_list, smooth = 10, 
                 buffer = 20, step_refine = 1, buffer_refine = 10, lam_refine = 0.1)
    cp_best, param_best, cp_best_cand = dcdp.fit(Y_train, Y_test)
    loc_error_dc[b] = cp_distance(cp_best, cp_truth)
    run_time_dc[b] = time.time() - start_time

print("avg loc error: {0}, avg time: {1}".format(loc_error_dc.mean(), run_time_dc.mean()))
print("best parameter: {0}".format(param_best))

---------- divide and conquer -----------
avg loc error: 466.0, avg time: 2.9182558059692383
best parameter: (0, 50)


In [125]:
print(cp_best)
print(cp_best_cand)

[966, 1624]
[ 952 1621]


Reasonable SNR

In [105]:
T = 4
Delta = 500
n = np.array([Delta] * T)
cp_truth = np.cumsum(n)[:T-1]

p = 5
theta = np.zeros((T, p, p))
theta[0] = np.eye(p)

rho = 0.3
theta[1] = np.eye(p)
for i in range(1, p):
    theta[1][i, i - 1] = rho
    theta[1][i - 1, i] = rho

rho = -0.3
theta[2] = np.eye(p)
for i in range(1, p):
    theta[2][i, i - 1] = rho
    theta[2][i - 1, i] = rho

theta[3] = np.eye(p)

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(theta[t] - theta[t - 1])**2)**0.5

nt, Y_train = generate_data_covariance(n, T, theta)
nt, Y_test = generate_data_covariance(n, T, theta)

In [115]:
grid_n = 100
repeat = 1
gamma_list = [20, 50, 100]
lam_list = [0]

B = 1
run_time_dc = np.zeros(B)
loc_error_dc = np.zeros(B)

print('---------- divide and conquer -----------')
for b in range(B):
    start_time = time.time()
    dcdp = dcdp_cv_random_covariance(grid_n, repeat, lam_list, gamma_list, smooth = 10, 
                 buffer = 20, step_refine = 1, buffer_refine = 10, lam_refine = 0.1)
    cp_best, param_best, cp_best_cand = dcdp.fit(Y_train, Y_test)
    loc_error_dc[b] = cp_distance(cp_best, cp_truth)
    run_time_dc[b] = time.time() - start_time

print("avg loc error: {0}, avg time: {1}".format(loc_error_dc.mean(), run_time_dc.mean()))
print("best parameter: {0}".format(param_best))

---------- divide and conquer -----------
avg loc error: 23.0, avg time: 2.955235242843628
best parameter: (0, 50)


In [116]:
print(cp_best)
print(cp_best_cand)

[499, 1000, 1523]
[ 484  985 1519]


In [117]:
cp = np.concatenate([[0], cp_truth, [len(Y_train)]])
sigma_path = dcdp.fit_with_cp(Y_train, cp)